# Recommendation System

In [441]:
#Libraries
import pandas as pd
import numpy as np
from random import *

In [442]:
#Load the dataset
data=pd.read_csv("facebook_combined.txt",sep=" ", header=None)

#Add column names
data.columns = ["node1", "node2"]

In [443]:
#Transform the graph to undirected
data2=pd.concat([data.node2,data.node1], axis=1)

#Rename the columns in order to merge the columns
data2.columns= ["node1", "node2"]
data=data.append(data2)

In [373]:
#Create a sample graph dataset
test_data = pd.DataFrame([[5, 2], 
                       [9, 3],
                       [9, 11],
                       [3, 6],
                       [4, 6],
                       [5, 7],
                       [1, 11],
                       [6, 2],
                       [7, 9],
                       [8, 9],
                       [5, 11],
                       [6, 7],
                       [6, 11],
                       [7, 6],
                       [2, 11],
                       [11,2],
                       [2, 5],
                       [6, 2],
                       [2, 7],
                       [7, 2]],
                      columns=["node1", "node2"])

#### Recommending friends using Common neighbors (friend-of-friend (FoF) method)

In [75]:
##### Create the function for Common neighbors

def friendOfFriend(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection=(len(set(friendships.get(target)).intersection(set(friendships.get(j)))))
            #print(intersection)
            inter[j]=intersection
   
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);
    

In [76]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
friendOfFriend(users, test_data, 5)

[6, 1, 9, 3, 4, 7, 8, 10, 11]

In [105]:
##### Test the code for the Original Dataset
users=list(range(0,4038))
friendOfFriend(users, data,30)

[67, 122, 186, 315, 25, 26, 272, 285, 332, 21]

###  Recommending friends using Jaccard coefficient

In [119]:
##### Create the function for Common neighbors using Jaccard coefficient

def JaccardCoefficient(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            # Create union
            union=len(set(friendships.get(target)).union(set(friendships.get(j))))
            # Check for No zero denominator
            if (union != 0) :
                inter[j]=len(set(friendships.get(target)).intersection(set(friendships.get(j))))/union

    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);
    

In [120]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
JaccardCoefficient(users, test_data, 5)

[6, 1, 11, 9, 3, 4, 7, 8, 10]

In [121]:
##### Test the code for the Original Dataset
users=list(range(0,4038))
JaccardCoefficient(users, data,30)

[276, 75, 88, 341, 133, 186, 297, 130, 79, 126]

### Recommending friends using Adamic and Adar function

In [94]:
def AdamicAdarFunction(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection = set(friendships.get(target)).intersection(set(friendships.get(j)))

            # Adamic and Adar score calculation
            sum = 0
            for k in intersection :
                if ( k in friendships.keys()) and (friendships[k] != []):
                    sum = sum+np.log(len(friendships[k]))

            if (sum != 0) :        
                inter[j]=1/sum
            else:
                inter[j]=0
            
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);

In [95]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
AdamicAdarFunction(users, test_data, 5)

[6, 11, 1, 3, 4, 7, 8, 9, 10]

In [109]:
##### Test the code for the Original Dataset
users=list(range(0,4038))
AdamicAdarFunction(users, data,30)

[2, 4, 6, 8, 11, 12, 14, 15, 17, 18]

### Evaluation of the recommendation system

In [446]:
# Create users list
users=list(range(100,4100,100))
data100=data[(data.node2%100 == 0) & (data.node1%100 == 0) & (data.node1 != 0) & (data.node2 != 0)]

#Run the functions
fofList=friendOfFriend(users, data100,100)
JaccardList=JaccardCoefficient(users, data100,100)
AdamicAdarList=AdamicAdarFunction(users, data100,100)

#Similarity Percentage of FoF and Jaccard
s1=len(set(fofList).intersection(set(JaccardList)))*10

#Similarity Percentage of FoF and Adamic and Adar
s2=len(set(fofList).intersection(set(AdamicAdarList)))*10

#Similarity Percentage of Jaccard and Adamic and Adar
s3=len(set(AdamicAdarList).intersection(set(JaccardList)))*10

#Average Similarity
avg_similarity=(s1+s2+s3)/3

### Forecast Recommendations

In [409]:
#Choose a pair F1 AND F2

def evaluationFunction(dataset,users,F1,F2):

    ####Remove the relationship
    
    #First we find the connection F1-F2
    l1=dataset[dataset.node2 == 5 ].index
    l2=dataset[dataset.node1 == 2 ].index
    rm1=set(l1).intersection(set(l2))
    
    #Then we find the connection F2-F1
    l1=dataset[dataset.node2 == 2 ].index
    l2=dataset[dataset.node1 == 5 ].index
    rm2=set(l1).intersection(set(l2))

    #We create the union
    rm=rm1.union(rm2)

    #Remove the elements of the set rm
    for i in rm:
        dataset=dataset.drop(i)
        
    ### Check if the Suggestion lists for all functions   
    
    if (F1 not in friendOfFriend(users, dataset,F2)): 
        return(None);

    if (F2 not in friendOfFriend(users, dataset,F1)): 
        return(None);

    if (F1 not in JaccardCoefficient(users, dataset,F2)): 
        return(None);

    if (F2 not in JaccardCoefficient(users, dataset,F1)): 
        return(None);
        
    if (F1 not in AdamicAdarFunction(users, dataset,F2)):
        return(None);

    if (F2 not in AdamicAdarFunction(users, dataset,F1)): 
        return(None);
    
    
    ###FoF (friend-of-friend)
        
    #Compute the recommendations for F1
    Friend1=friendOfFriend(users, dataset,F1).index(F2)+1
    
    #Compute the resommentdations for F2
    Friend2=friendOfFriend(users, dataset,F2).index(F1)+1
    

    ####Compute the score
    scoreFoF=(Friend1+Friend2)/2
    
    
    ###Jaccard
    #Compute the recommendations for F1
    Friend1=JaccardCoefficient(users, dataset,F1).index(F2)+1
    
    #Compute the resommentdations for F2
    Friend2=JaccardCoefficient(users, dataset,F2).index(F1)+1
    
    ####Check if either of these does not exist

    ####Compute the score
    scoreJaccard=(Friend1+Friend2)/2
    
    
    ###AdamicAdar
    #Compute the recommendations for F1
    Friend1=AdamicAdarFunction(users, dataset,F1).index(F2)+1
    
    #Compute the resommentdations for F2
    Friend2=AdamicAdarFunction(users, dataset,F2).index(F1)+1
    
    ####Check if either of these does not exist

    ####Compute the score
    scoreAdamicAdar=(Friend1+Friend2)/2
    
    return(scoreFoF,scoreJaccard,scoreAdamicAdar);

In [410]:
#Function for iterations
def finalScore(dataset,users, n):
    eval_scores=[]
    for l in list(range(0,n)):
        friends=sample(users,2) #take 2 random users
        if (evaluationFunction(dataset,users,friends[0],friends[1]) != None):
            eval_scores.append(evaluationFunction(dataset,users,friends[0],friends[1]))
        
    return(eval_scores)       

In [440]:
#Create the scores!!
users=[1,2,3,4,5,6,7,8,9,10,11]
scores=np.mean(finalScore(test_data,users,6),axis=0)

print("FoF score is",scores[0])
print("Jaccard score is",scores[1])
print("Adamic Adar score is",scores[2])

FoF score is 3.9
Jaccard score is 4.0
Adamic Adar score is 4.4


In [445]:
users=list(range(0,4038))
finalScore(data,users,10)

[]

In [462]:
users=list(range(100,4100,100))
data100=data[(data.node2%100 == 0) & (data.node1%100 == 0) & (data.node1 != 0) & (data.node2 != 0)]
finalScore(data100,users,1)

[]